In [11]:
import pandas as pd
import numpy as np
import spacy
from sklearn import (
    feature_extraction,
    linear_model,
    metrics,
    model_selection,
    preprocessing,
)

In [3]:
#!pip install spacy
#!python -m spacy download pt_core_news_sm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/DataScience/DS-Tera/ProjetoBNCC/data/data_preprocess_basic.csv')

In [3]:
data.head()

,id,questoes,tipoQuestoes,topico,materia,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en,questoes_tokens,questoes_stemm,questoes_lemma
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,"['base', 'conhecimentos', 'sobre', 'compositor...","['bas', 'conhec', 'sobr', 'compositor', 'john'...","['base', 'conhecimentos', 'sobre', 'compositor..."
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...,"['achar', 'número', 'somado', 'igual', 'invers...","['achar', 'númer', 'som', 'igual', 'invers', '...","['achar', 'número', 'somado', 'igual', 'invers..."
2,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,"['chuva', 'ácida', 'fenômeno', 'provocado', 'p...","['chuv', 'ácid', 'fenômen', 'provoc', 'poluiçã...","['chuva', 'ácida', 'fenômeno', 'provocado', 'p..."
3,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,História,Fundamental II,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr. do livro didático.,pagina nr do livro didático,pagina nr livro didático,pagina nr livro didático,"['pagina', 'nr', 'livro', 'didático']","['pagin', 'nr', 'livr', 'didát']","['pagina', 'nr', 'livro', 'didático']"
4,2525602,<p>O que &eacute; CHIAROSCURO?</p>,4,História da Arte: Arte Barroca,Arte,Médio & Pré-Vestibular,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro?,o que é chiaroscuro,chiaroscuro,chiaroscuro,['chiaroscuro'],['chiaroscur'],['chiaroscuro']


In [4]:
data_postagg = data.copy()

POS tagging - etiqueta partes da fala, serve para analisar e identificar as diferentes classes gramaticais em um texto

In [5]:
#pipeline para lemmatizing e POS tagging das questoes usando spacy

nlp = spacy.load('pt_core_news_sm')

tokens = []
lemma = []
pos = []

for doc in nlp.pipe(data_postagg['questoes_stopw_en'].astype('unicode').values, batch_size=50):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

data_postagg['questoes_tokens_s'] = tokens
data_postagg['questoes_lemma_s'] = lemma
data_postagg['questoes_pos_s'] = pos

In [6]:
data_postagg.head()

,id,questoes,tipoQuestoes,topico,materia,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en,questoes_tokens,questoes_stemm,questoes_lemma,questoes_tokens_s,questoes_lemma_s,questoes_pos_s
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,"['base', 'conhecimentos', 'sobre', 'compositor...","['bas', 'conhec', 'sobr', 'compositor', 'john'...","['base', 'conhecimentos', 'sobre', 'compositor...","[base, conhecimentos, sobre, compositor, john,...","[base, conhecimento, sobrar, compositor, john,...","[NOUN, NOUN, ADP, NOUN, PROPN, PROPN, PROPN, P..."
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...,"['achar', 'número', 'somado', 'igual', 'invers...","['achar', 'númer', 'som', 'igual', 'invers', '...","['achar', 'número', 'somado', 'igual', 'invers...","[achar, número, somado, igual, inverso, equaçõ...","[achar, número, somar, igual, inverso, equação...","[VERB, NOUN, VERB, ADJ, PROPN, PROPN, VERB, NOUN]"
2,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,"['chuva', 'ácida', 'fenômeno', 'provocado', 'p...","['chuv', 'ácid', 'fenômen', 'provoc', 'poluiçã...","['chuva', 'ácida', 'fenômeno', 'provocado', 'p...","[chuva, ácida, fenômeno, provocado, poluição, ...","[chuva, ácido, fenômeno, provocar, poluição, a...","[NOUN, ADJ, ADJ, VERB, NOUN, ADJ, ADP, PROPN, ..."
3,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,História,Fundamental II,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr. do livro didático.,pagina nr do livro didático,pagina nr livro didático,pagina nr livro didático,"['pagina', 'nr', 'livro', 'didático']","['pagin', 'nr', 'livr', 'didát']","['pagina', 'nr', 'livro', 'didático']","[pagina, nr, livro, didático]","[paginar, nr, livrar, didático]","[VERB, ADP, NOUN, ADJ]"
4,2525602,<p>O que &eacute; CHIAROSCURO?</p>,4,História da Arte: Arte Barroca,Arte,Médio & Pré-Vestibular,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro?,o que é chiaroscuro,chiaroscuro,chiaroscuro,['chiaroscuro'],['chiaroscur'],['chiaroscuro'],[chiaroscuro],[chiaroscuro],[NOUN]


In [15]:
# questoes lemmatizadas usando Spacy

data_postagg['questoes_lemma_s'] = data_postagg['questoes_lemma_s'].apply(" ".join)
data_postagg['questoes_lemma_s']

0        base conhecimento sobrar compositor john cage ...
1        achar número somar igual inverso equação repre...
2        chuva ácido fenômeno provocar poluição atmosfé...
3                               paginar nr livrar didático
4                                              chiaroscuro
                               ...                        
82290    ler reportagem responder questão sobrar pergun...
82291    responder questão ler trecho contar prefácio h...
82292                                     pai surgir skate
82293    macho macho mosquito gênero aedes viver local ...
82294    observar tabelar ver quanto lanche vendido trê...
Name: questoes_lemma_s, Length: 82295, dtype: object

In [42]:
data_postagg.head()

,id,questoes,tipoQuestoes,topico,materia,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en,questoes_tokens,questoes_stemm,questoes_lemma,questoes_tokens_s,questoes_lemma_s,questoes_pos_s
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,"['base', 'conhecimentos', 'sobre', 'compositor...","['bas', 'conhec', 'sobr', 'compositor', 'john'...","['base', 'conhecimentos', 'sobre', 'compositor...","[base, conhecimentos, sobre, compositor, john,...",base conhecimento sobrar compositor john cage ...,"[NOUN, NOUN, ADP, NOUN, PROPN, PROPN, PROPN, P..."
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...,"['achar', 'número', 'somado', 'igual', 'invers...","['achar', 'númer', 'som', 'igual', 'invers', '...","['achar', 'número', 'somado', 'igual', 'invers...","[achar, número, somado, igual, inverso, equaçõ...",achar número somar igual inverso equação repre...,"[VERB, NOUN, VERB, ADJ, PROPN, PROPN, VERB, NOUN]"
2,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,"['chuva', 'ácida', 'fenômeno', 'provocado', 'p...","['chuv', 'ácid', 'fenômen', 'provoc', 'poluiçã...","['chuva', 'ácida', 'fenômeno', 'provocado', 'p...","[chuva, ácida, fenômeno, provocado, poluição, ...",chuva ácido fenômeno provocar poluição atmosfé...,"[NOUN, ADJ, ADJ, VERB, NOUN, ADJ, ADP, PROPN, ..."
3,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,História,Fundamental II,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr. do livro didático.,pagina nr do livro didático,pagina nr livro didático,pagina nr livro didático,"['pagina', 'nr', 'livro', 'didático']","['pagin', 'nr', 'livr', 'didát']","['pagina', 'nr', 'livro', 'didático']","[pagina, nr, livro, didático]",paginar nr livrar didático,"[VERB, ADP, NOUN, ADJ]"
4,2525602,<p>O que &eacute; CHIAROSCURO?</p>,4,História da Arte: Arte Barroca,Arte,Médio & Pré-Vestibular,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro?,o que é chiaroscuro,chiaroscuro,chiaroscuro,['chiaroscuro'],['chiaroscur'],['chiaroscuro'],[chiaroscuro],chiaroscuro,[NOUN]


Bag of Words vectorizer

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizerBOW = CountVectorizer()
XBOW = vectorizerBOW.fit_transform(data_postagg['questoes_lemma_s'])
XBOW

<82295x88529 sparse matrix of type '<class 'numpy.int64'>'
	with 2274643 stored elements in Compressed Sparse Row format>

TF-IDF vectorizer

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerTF = TfidfVectorizer()
XTF = vectorizerTF.fit_transform(data_postagg['questoes_lemma_s'])
XTF

<82295x88529 sparse matrix of type '<class 'numpy.float64'>'
	with 2274643 stored elements in Compressed Sparse Row format>

Dividindo os dados para treino e teste

In [43]:
from sklearn.model_selection import train_test_split

y = data_postagg['etapaEnsino']

X = data_postagg['questoes_lemma_s']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

XBOW_train, XBOW_test, y_train, y_test = train_test_split(XBOW, y, random_state=42)

XTF_train, XTF_test, y_train, y_test = train_test_split(XTF, y, random_state=42)

Modelo Regressão Linear

In [49]:
logreg = linear_model.LogisticRegression(class_weight="balanced")
logreg.fit(XTF_train, y_train)
y_pred_class = logreg.predict(XTF_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [50]:
print(metrics.classification_report(y_test, y_pred_class))

                        precision    recall  f1-score   support

         Fundamental I       0.64      0.79      0.71      4382
        Fundamental II       0.66      0.62      0.64      7992
Médio & Pré-Vestibular       0.76      0.70      0.73      8200

              accuracy                           0.69     20574
             macro avg       0.68      0.70      0.69     20574
          weighted avg       0.69      0.69      0.69     20574

